In [ ]:
!pip install numpy pandas matplotlib seaborn wordcloud pillow matplotlib-venn spacy nltk


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')


In [ ]:
import numpy as np
import pandas as pd
#misc
import gc
import time
import warnings

#stats
from scipy import sparse
import scipy.stats as ss

#viz
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from wordcloud import WordCloud ,STOPWORDS
from PIL import Image
import matplotlib_venn as venn

import numpy as np
import pandas as pd
#misc
import gc
import time
import warnings

#stats
from scipy import sparse
import scipy.stats as ss

#viz
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from wordcloud import WordCloud ,STOPWORDS
from PIL import Image
import matplotlib_venn as venn

#nlp
import string
import re    #for regex
import nltk
from nltk.corpus import stopwords
import spacy
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
# Tweet tokenizer does not split at apostophes which is what we want
from nltk.tokenize import TweetTokenizer


#settings
start_time=time.time()
color = sns.color_palette()
sns.set_style("dark")
eng_stopwords = set(stopwords.words("english"))
warnings.filterwarnings("ignore")

%matplotlib inline

In [ ]:
file_path = '/content/drive/MyDrive/toxic_comment/train.csv'
train = pd.read_csv(file_path)
train.head(10)

In [ ]:
file_path = '/content/drive/MyDrive/toxic_comment/test.csv'
test= pd.read_csv(file_path)
test.head(10)

In [ ]:

label_col = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
if 'is_clean' not in train.columns:
    train['is_clean'] = (train[label_col].sum(axis=1) == 0).astype(int)

# Total characters
train['total_len'] = train['comment_text'].apply(len)
test['total_len'] = test['comment_text'].apply(len)

# Sentence count
train['sent_count'] = train["comment_text"].apply(lambda x: len(re.findall("\n", str(x))) + 1)
test['sent_count'] = test["comment_text"].apply(lambda x: len(re.findall("\n", str(x))) + 1)

# Word count
train['word_count'] = train["comment_text"].apply(lambda x: len(str(x).split()))
test['word_count'] = test["comment_text"].apply(lambda x: len(str(x).split()))

# Plot KDEs
plt.figure(figsize=(18, 6))
plt.suptitle("Are longer comments more toxic?", fontsize=18)

# Characters
plt.subplot(131)
sns.kdeplot(train[train.is_clean == 0]['total_len'], label="UnClean", shade=True, color='r')
sns.kdeplot(train[train.is_clean == 1]['total_len'], label="Clean")
plt.legend()
plt.ylabel('Density', fontsize=12)
plt.xlabel('# of Chars', fontsize=12)

# Words
plt.subplot(132)
sns.kdeplot(train[train.is_clean == 0]['word_count'], label="UnClean", shade=True, color='r')
sns.kdeplot(train[train.is_clean == 1]['word_count'], label="Clean")
plt.legend()
plt.xlabel('# of Words', fontsize=12)

# Sentences
plt.subplot(133)
sns.kdeplot(train[train.is_clean == 0]['sent_count'], label="UnClean", shade=True, color='r')
sns.kdeplot(train[train.is_clean == 1]['sent_count'], label="Clean")
plt.legend()
plt.xlabel('# of Sentences', fontsize=12)

plt.tight_layout()
plt.show()

most comments are having less than 25 sentences & less than 250 words

unclean comments are having more no.of words in less no.of sentences.

The distrubution plots of clean & unclean of all three plots are very much overlapping with each others, indicating these features are going to be less significant in differentiating them.

In [ ]:
import string


# Ensure is_clean exists (0 = toxic, 1 = clean)
label_col = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
if 'is_clean' not in train.columns:
    train['is_clean'] = (train[label_col].sum(axis=1) == 0).astype(int)

# Capital letters count
train['capitals'] = train['comment_text'].apply(lambda x: sum(1 for c in str(x) if c.isupper()))
test['capitals'] = test['comment_text'].apply(lambda x: sum(1 for c in str(x) if c.isupper()))

# Punctuation count
train['punct_count'] = train['comment_text'].apply(lambda x: sum(1 for c in str(x) if c in string.punctuation))
test['punct_count'] = test['comment_text'].apply(lambda x: sum(1 for c in str(x) if c in string.punctuation))

# Smilies count
smilies = (':-)', ':)', ';-)', ';)')
train['smilies_count'] = train['comment_text'].apply(lambda comment: sum(str(comment).count(s) for s in smilies))
test['smilies_count'] = test['comment_text'].apply(lambda comment: sum(str(comment).count(s) for s in smilies))

# Plotting
plt.figure(figsize=(18, 6))
plt.suptitle("Does the Presence of Special Characters Vary with Toxicity?", fontsize=18)

# Capitals
plt.subplot(131)
sns.kdeplot(train[train.is_clean == 0]['capitals'], label="Toxic", shade=True, color='r')
sns.kdeplot(train[train.is_clean == 1]['capitals'], label="Clean")
plt.legend()
plt.ylabel('Density', fontsize=12)
plt.xlabel('# of Capital Letters', fontsize=12)

# Punctuations
plt.subplot(132)
sns.kdeplot(train[train.is_clean == 0]['punct_count'], label="Toxic", shade=True, color='r')
sns.kdeplot(train[train.is_clean == 1]['punct_count'], label="Clean")
plt.legend()
plt.xlabel('# of Punctuations', fontsize=12)

# Smilies
plt.subplot(133)
sns.kdeplot(train[train.is_clean == 0]['smilies_count'], label="Toxic", shade=True, color='r')
sns.kdeplot(train[train.is_clean == 1]['smilies_count'], label="Clean")
plt.legend()
plt.xlabel('# of Smilies', fontsize=12)

plt.tight_layout()
plt.show()

presence of captial letters is more in case of unclean comments, but the distrbutions are overlapping making it a difficult feature for models to extract information.

most of the clean comments are having punctuations less than 100 while for unclean comments it spread to max of 5000 punctuations.

no.of smilies in unclean v/s clean comments is very much similar and unclean comments are having more comments with no.of smilies = 1.

In [ ]:
# Unique word count
train['unique_word_count'] = train["comment_text"].apply(lambda x: len(set(str(x).split())))
test['unique_word_count'] = test["comment_text"].apply(lambda x: len(set(str(x).split())))

# Unique ratio
train['unique_word_percent'] = (train['unique_word_count'] / train['word_count']) * 100
test['unique_word_percent'] = (test['unique_word_count'] / test['word_count']) * 100

# ---------- Plotting ------------
plt.figure(figsize=(15, 5))
plt.suptitle("Comments with Less-Unique-Words (Spam) vs Toxicity?", fontsize=18)

# KDE plot for unique word percentage
plt.subplot(121)
plt.title("% of Unique Words in Comments")
sns.kdeplot(train[train.is_clean == 0]['unique_word_percent'], label="Toxic", shade=True, color='r')
sns.kdeplot(train[train.is_clean == 1]['unique_word_percent'], label="Clean")
plt.legend()
plt.ylabel('Density', fontsize=12)
plt.xlabel('Percent Unique Words', fontsize=12)

# Violin plot for comments with <25% unique words
plt.subplot(122)
sns.violinplot(
    y='unique_word_count', x='is_clean',
    data=train[train['unique_word_percent'] < 25],
    split=True, inner="quart"
)
plt.xlabel('is_Clean', fontsize=12)
plt.ylabel('# of Unique Words', fontsize=12)
plt.title("# Unique Words vs Toxicity")

plt.tight_layout()
plt.show()


There is a wide spread area for unclean points in the unique word percentage range of 1-10%, Interesting there are clean comments as well with lesser number of unique words.

This feature seems carry some significance especially incase of sentences with less unique words.

lets once see how text in clean-spam & unclean-spam comments look like

In [ ]:
## lets have a look how clean & unclean spam comment looks like

print("Clean Spam example:")
print(train[train['unique_word_percent'] < 10][train['is_clean'] == 1].comment_text.iloc[3])
print('-'*50)
print("Toxic Spam example:")
print(train[train['unique_word_percent'] < 10][train['is_clean'] == 0].comment_text.iloc[25])

we definitely need to penalize the over repeating words (or) there will be chance for models to learn as words like 'George Bush' as indicator as toxicity. so comments with lesser unique words should be preprocessed carefully.





##Classical models (TF-IDF) cleaner

In [ ]:
import re
import string
from nltk.corpus import stopwords

eng_stopwords = set(stopwords.words("english"))

CONTRACTIONS = {
    "can't": "cannot", "won't": "will not", "i'm": "i am", "it's": "it is",
    "don't": "do not", "didn't": "did not", "you're": "you are",
    "they're": "they are", "isn't": "is not", "aren't": "are not",
    # add more as needed
}
def expand_contractions(text):
    def replace(match):
        return CONTRACTIONS.get(match.group(0).lower(), match.group(0))
    pattern = re.compile(r'\b(' + '|'.join(map(re.escape, CONTRACTIONS.keys())) + r')\b', flags=re.IGNORECASE)
    return pattern.sub(replace, text)

def clean_text(text):
    if not isinstance(text, str):
        text = str(text)
    text = text.lower()
    text = expand_contractions(text)
    text = re.sub(r'http\S+|www\.\S+', ' ', text)              # remove urls
    text = re.sub(r'\d+', ' ', text)                           # remove numbers (optional)
    text = text.translate(str.maketrans('', '', string.punctuation))  # remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()                   # normalize spaces
    # remove stopwords
    tokens = [t for t in text.split() if t not in eng_stopwords]
    return ' '.join(tokens)

In [ ]:

train['clean_text'] = train['comment_text'].apply(clean_text)

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    max_features=50_000,

    stop_words=None
)

X_tfidf = tfidf.fit_transform(train['clean_text'])

feature_names = tfidf.get_feature_names_out()
print("TF‑IDF shape:", X_tfidf.shape)



## TF-IDF Vectorizer
TF - Term Frequency -- Count of the words(Terms) in the text corpus (same of Count Vect)
IDF - Inverse Document Frequency -- Penalizes words that are too frequent. We can think of this as regularization

In [ ]:
def get_topn_tfidf_feat_byClass(X_tfidf, y_train, feature_names, labels, topn):
    feat_imp_dfs = {}

    for label in labels:
        # Get indices of rows where label is true
        label_ids = y_train.index[y_train[label] == 1]
        # Get subset of rows
        label_rows = X_tfidf[label_ids].toarray()
        # Calculate mean feature importance
        feat_imp = label_rows.mean(axis=0)
        # Sort by column dimension and get topn feature indices
        topn_ids = np.argsort(feat_imp)[::-1][:topn]
        # Combine tfidf value with feature name
        topn_features = [(feature_names[i], feat_imp[i]) for i in topn_ids]
        # Save to dataframe
        topn_df = pd.DataFrame(topn_features, columns=['word_feature', 'tfidf_value'])
        feat_imp_dfs[label] = topn_df

    return feat_imp_dfs

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create the vectorizer
tfidf = TfidfVectorizer(
    ngram_range=(1, 1),  # unigrams
    max_features=50000,
    stop_words='english'
)

# Fit and transform the training text
X_tfidf = tfidf.fit_transform(train['comment_text'])

# Extract feature names
feature_names = tfidf.get_feature_names_out()

In [ ]:
label_col = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
top_features = get_topn_tfidf_feat_byClass(X_tfidf, train[label_col], feature_names, label_col, topn=20)
print(top_features['toxic'])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Sample TF-IDF
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_tfidf = vectorizer.fit_transform(train['comment_text'])

# Get feature names
feature_names = vectorizer.get_feature_names_out()

# Labels
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# Get top 20 features per label
top_features = get_topn_tfidf_feat_byClass(X_tfidf, train[labels], feature_names, labels, topn=20)

# Example: Print top 20 words for "toxic"
print(top_features['toxic'])


# Important unigrams by class visuals

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Define label columns
label_col = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# TF-IDF Vectorizer
tfidf = TfidfVectorizer(
    ngram_range=(1, 1), min_df=100,
    strip_accents='unicode', analyzer='word',
    use_idf=True, smooth_idf=True, sublinear_tf=True,
    stop_words='english'
)

X_unigrams = tfidf.fit_transform(train['comment_text'])
print("Shape of TF-IDF Matrix:", X_unigrams.shape)
print("Number of Features:", len(tfidf.get_feature_names_out()))

feature_names = np.array(tfidf.get_feature_names_out())

# Get top TF-IDF features per class
imp_dfs = get_topn_tfidf_feat_byClass(X_unigrams, train[label_col], feature_names, label_col, topn=10)

# Plotting - Horizontal colorful bars
plt.figure(figsize=(15, 10))
for i, label in enumerate(label_col):
    plt.subplot(3, 2, i + 1)
    sns.barplot(
        x='tfidf_value',
        y='word_feature',
        data=imp_dfs[label].head(10),
        palette="Paired"  # colorful palette
    )
    plt.title(f"Important UniGrams for the class: {label}", fontsize=12)
    plt.xlabel("TF-IDF Value")
    plt.ylabel("")

plt.tight_layout()
plt.show()

few words like fuck seems to be in every class, but again as this is multi-label classification (multiple tags for each comment) there will be that overlapping.

especially threat class is standing apart with words like kill, die, death.

Interestingly, due to high tf-idf value, word wikipedia has stands in top10 features for the toxic class. which model should not learn.

#Important bigrams by class visuals

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# TF-IDF for Bigrams
tfidf = TfidfVectorizer(
    ngram_range=(2, 2), min_df=100,
    strip_accents='unicode', analyzer='word',
    use_idf=True, smooth_idf=True, sublinear_tf=True,
    stop_words='english'
)

X_bigrams = tfidf.fit_transform(train['comment_text'])
print("Shape of TF-IDF Matrix:", X_bigrams.shape)
print("Number of Features:", len(tfidf.get_feature_names_out()))

feature_names = np.array(tfidf.get_feature_names_out())

# Top 10 bigrams per class
imp_dfs = get_topn_tfidf_feat_byClass(X_bigrams, train[label_col], feature_names, label_col, topn=10)

# Plotting
plt.figure(figsize=(15, 12))
for i, label in enumerate(label_col):
    plt.subplot(3, 2, i + 1)
    sns.barplot(
        x='tfidf_value',
        y='word_feature',
        data=imp_dfs[label].head(10),
        palette="Set2"  # Colorful bars
    )
    plt.title(f"Important BiGrams for the class: {label}")
    plt.xlabel("TF-IDF Score")
    plt.ylabel("")
    plt.tight_layout()

plt.show()


the top 10 tfidf features in case are not much differentiating, every class are having almost same phrases.
remember, the comment_text is completely unprocessed for now.

There is high chance for Names of persons like famous names george bush, mitt romney (if mentioned repeatedly in the spam) to appear in the bigram features for any class. To avoid this we

can preprocess spam comments (comments with < 25% unique words) differently like removing usernames/person names from the comments using regular expressions or POS tags.

#Train and test split

In [ ]:
from sklearn.model_selection import train_test_split
train['clean_text'] = train['comment_text'].apply(clean_text)
test['clean_text']  = test['comment_text'].apply(clean_text)

X_train, X_val, y_train, y_val = train_test_split(
    train['clean_text'],               # <‑‑ now split on cleaned text
    train[label_col],
    test_size=0.2,
    random_state=2019
)

tfidf = TfidfVectorizer(max_features=50_000, ngram_range=(1, 2))
X_train_tf = tfidf.fit_transform(X_train)
X_val_tf   = tfidf.transform(X_val)
X_test_tf  = tfidf.transform(test['clean_text'])


In [ ]:
# from sklearn.model_selection import train_test_split

# # Define label columns
# label_col = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# # Split data
# X_train, X_val, y_train, y_val = train_test_split(
#     train['comment_text'],
#     train[label_col],
#     test_size=0.2,
#     random_state=2019
# )

# X_test = test['comment_text']

# print('Train data size:', len(X_train))
# print('Validation data size:', len(X_val))
# print('Test data size:', len(X_test))


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=9,
    strip_accents='unicode',
    analyzer='word',
    use_idf=True,
    smooth_idf=True,
    sublinear_tf=True,
    stop_words='english'
)

X_train_tf = tfidf.fit_transform(X_train)
X_val_tf = tfidf.transform(X_val)
X_test_tf = tfidf.transform(X_test)

print('Final Data dimensions after transformations:', X_train_tf.shape, y_train.shape, X_val_tf.shape, y_val.shape)


Modelling: Baselines

Generally, I will begin with least complex models & will move to complex models based on their time/space complexity.

1. Naive Bayes
Naive Bayes is well know for its Text Sentiment Classification tasks. It is also simple to interpret the model so I am starting with it.

If works well, added advantages includes speed - as worst case complexity is O(Nd) is comparatively low.

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
model = OneVsRestClassifier(MultinomialNB(), n_jobs = -1)
model.fit(X_train_tf, y_train)
print('model: Naive Bayes')
print('mean ROC-AUC on train set:', roc_auc_score(y_train, model.predict_proba(X_train_tf)))
y_pred_nb = model.predict_proba(X_val_tf)
print('mean ROC-AUC on validation set:', roc_auc_score(y_val, y_pred_nb))

In [ ]:
model = MultinomialNB()

train_rocs = []
valid_rocs = []

preds_train = np.zeros(y_train.shape)
preds_valid = np.zeros(y_val.shape)
preds_test = np.zeros((len(test), len(label_col)))
print('model: Naive Bayes')
for i, label_name in enumerate(label_col):
    print('\nClass:= '+label_name)
    # fit
    model.fit(X_train_tf,y_train[label_name])

    # train
    preds_train[:,i] = model.predict_proba(X_train_tf)[:,1]
    train_roc_class = roc_auc_score(y_train[label_name],preds_train[:,i])
    print('Train ROC AUC:', train_roc_class)
    train_rocs.append(train_roc_class)

    # valid
    preds_valid[:,i] = model.predict_proba(X_val_tf)[:,1]
    valid_roc_class = roc_auc_score(y_val[label_name],preds_valid[:,i])
    print('Valid ROC AUC:', valid_roc_class)
    valid_rocs.append(valid_roc_class)

    # test predictions
    preds_test[:,i] = model.predict_proba(X_test_tf)[:,1]

print('\nmean column-wise ROC AUC on Train data: ', np.mean(train_rocs))
print('mean column-wise ROC AUC on Val data:', np.mean(valid_rocs))

And the results of both implementations are excatly same..
Naive Bayes performed pretty well in its zone with 0.9047...
This looks like a good score only until we ecperiment with other models...as they might score a way higher lets see...how linear models like Logistic regression & linear SVM perform with default parameters

From here on I will be using the onevsrest classifier.

2. Logisitc Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
model = OneVsRestClassifier(LogisticRegression(), n_jobs = -1)
model.fit(X_train_tf, y_train)
print('model: Logistic Regression')
print('mean ROC-AUC on train set:', roc_auc_score(y_train, model.predict_proba(X_train_tf)))
y_pred_log = model.predict_proba(X_val_tf)
print('mean ROC-AUC on validation set:', roc_auc_score(y_val, y_pred_log))

3. Linear SVM

Linear SVM will not support probability prediction as hinge loss is unstable, the way around is using a calibration classifer wrapper and it wont support OnevsRest classifers. so we I am using manual python loop again.

Notice that I fitted a new CalibratedClassifierCV wrapper afer fitting the LinearSVC classifier.

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

model = LinearSVC()

train_rocs = []
valid_rocs = []

preds_train = np.zeros(y_train.shape)
preds_valid = np.zeros(y_val.shape)
preds_test = np.zeros((len(test), len(label_col)))
print('model: Linear SVM')
for i, label_name in enumerate(label_col):
    print('\nClass:= '+label_name)

    # fit
    model.fit(X_train_tf,y_train[label_name])

    # calibration classifier fit
    model = CalibratedClassifierCV(model, cv = 'prefit')
    model.fit(X_train_tf, y_train[label_name])

    # train
    preds_train[:,i] = model.predict_proba(X_train_tf)[:,1]
    train_roc_class = roc_auc_score(y_train[label_name],preds_train[:,i])
    print('Train ROC AUC:', train_roc_class)
    train_rocs.append(train_roc_class)

    # valid
    preds_valid[:,i] = model.predict_proba(X_val_tf)[:,1]
    valid_roc_class = roc_auc_score(y_val[label_name],preds_valid[:,i])
    print('Valid ROC AUC:', valid_roc_class)
    valid_rocs.append(valid_roc_class)

    # test predictions
    preds_test[:,i] = model.predict_proba(X_test_tf)[:,1]

print('\nmean column-wise ROC AUC on Train data: ', np.mean(train_rocs))
print('mean column-wise ROC AUC on Val data:', np.mean(valid_rocs))

Performance of Linear Models:

Both models Logistic & SVM are performing better than naive bayes models.

Although Logistic is better with default parameters, I need to experiment & see if fine tuning helps Linear SVM.

Moreover, these models are interpretable & not very complex in terms of time & space required.



##ML experimenting and tracking using weight and biases


In [ ]:
import wandb
wandb.login()


In [ ]:
# --- Install and import dependencies ---
!pip install wandb scikit-learn matplotlib seaborn -q

import wandb
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc, confusion_matrix, roc_auc_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

# --- Helper function to log metrics and plots ---
def log_model_to_wandb(model_name, model, X_train, y_train, X_val, y_val):
    # Start a new run
    wandb.init(project="toxic_comment_classification", name=model_name)

    # Store metrics for each class (multi-label)
    class_rocs_train = []
    class_rocs_val = []
    preds_train = np.zeros(y_train.shape)
    preds_val = np.zeros(y_val.shape)

    for i, label_name in enumerate(y_train.columns):
        # Fit model per class
        model.fit(X_train, y_train[label_name])
        preds_train[:, i] = model.predict_proba(X_train)[:, 1]
        preds_val[:, i] = model.predict_proba(X_val)[:, 1]

        # Compute ROC AUC for train/val
        roc_train = roc_auc_score(y_train[label_name], preds_train[:, i])
        roc_val = roc_auc_score(y_val[label_name], preds_val[:, i])
        class_rocs_train.append(roc_train)
        class_rocs_val.append(roc_val)

        # Log confusion matrix
        cm = confusion_matrix(y_val[label_name], (preds_val[:, i] > 0.5).astype(int))
        fig, ax = plt.subplots()
        sns.heatmap(cm, annot=True, fmt="d", ax=ax, cmap="Blues")
        ax.set_title(f"Confusion Matrix - {label_name}")
        wandb.log({f"Confusion Matrix/{label_name}": wandb.Image(fig)})
        plt.close(fig)

        # Log ROC curve
        fpr, tpr, _ = roc_curve(y_val[label_name], preds_val[:, i])
        fig, ax = plt.subplots()
        ax.plot(fpr, tpr, label=f"{label_name} (AUC={roc_val:.3f})")
        ax.plot([0, 1], [0, 1], 'k--')
        ax.set_title(f"ROC Curve - {label_name}")
        ax.set_xlabel("False Positive Rate")
        ax.set_ylabel("True Positive Rate")
        ax.legend()
        wandb.log({f"ROC Curve/{label_name}": wandb.Image(fig)})
        plt.close(fig)

    # Log mean ROC AUC
    wandb.log({
        "mean_roc_auc_train": np.mean(class_rocs_train),
        "mean_roc_auc_val": np.mean(class_rocs_val)
    })

    wandb.finish()


# --- Run for each model ---
# 1. Naive Bayes
nb_model = MultinomialNB()
log_model_to_wandb("Naive_Bayes", nb_model, X_train_tf, y_train, X_val_tf, y_val)

# 2. Logistic Regression
lr_model = LogisticRegression(max_iter=500)
log_model_to_wandb("Logistic_Regression", lr_model, X_train_tf, y_train, X_val_tf, y_val)

# 3. Linear SVM (with calibration for predict_proba)
svm_model = CalibratedClassifierCV(LinearSVC())
log_model_to_wandb("Linear_SVM", svm_model, X_train_tf, y_train, X_val_tf, y_val)


In [ ]:
# --- Install dependencies ---
!pip install wandb matplotlib seaborn pandas -q

import wandb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc

# --- W&B Project Info ---
PROJECT_PATH = "fareedahab-infineon-technologies/toxic_comment_classification"

# --- Pull Runs from W&B ---
api = wandb.Api()
runs = api.runs(PROJECT_PATH)

# --- Create Leaderboard DataFrame ---
data = []
for run in runs:
    model_name = run.name
    auc_train = run.summary.get('mean_roc_auc_train', None)
    auc_val = run.summary.get('mean_roc_auc_val', None)
    data.append({"Model": model_name, "Train AUC": auc_train, "Validation AUC": auc_val})

leaderboard = pd.DataFrame(data)
print("\nLeaderboard Table:\n", leaderboard)

# Save leaderboard as image
fig, ax = plt.subplots(figsize=(6, 2))
ax.axis('tight')
ax.axis('off')
ax.table(cellText=leaderboard.values, colLabels=leaderboard.columns, loc='center')
plt.savefig("wandb_leaderboard.png", bbox_inches='tight')
plt.show()

# --- Combined ROC Curves ---
def plot_combined_roc(y_val, preds_dict, label_name):
    """
    preds_dict: dictionary of {model_name: predictions_for_class}
    y_val: true labels (e.g., y_val['toxic'])
    """
    plt.figure(figsize=(8, 6))
    for model_name, preds in preds_dict.items():
        fpr, tpr, _ = roc_curve(y_val, preds)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f"{model_name} (AUC={roc_auc:.3f})")
    plt.plot([0, 1], [0, 1], 'k--')
    plt.title(f"ROC Curve - {label_name}")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend(loc="lower right")
    plt.savefig(f"roc_combined_{label_name}.png", bbox_inches='tight')
    plt.show()

# Example for the "toxic" class:
# preds_dict = {
#     "Naive Bayes": preds_val_nb[:, 0],
#     "Logistic Regression": preds_val_lr[:, 0],
#     "Linear SVM": preds_val_svm[:, 0]
# }
# plot_combined_roc(y_val['toxic'], preds_dict, label_name="toxic")

